
# 線形代数

:label: `sec_linear-algebra`

これまでに、データセットをテンソルにロードし、基本的な数学的演算でこれらのテンソルを操作できるようになりました。洗練されたモデルの構築を開始するには、線形代数のツールもいくつか必要です。このセクションでは、スカラー演算から始まり行列の乗算まで、最も重要な概念を優しく紹介します。


In [1]:
import torch


## スカラー

日常的な数学のほとんどは、一度に 1 つずつ数値を操作することで構成されます。正式には、これらの値を*スカラー と*呼びます。たとえば、パロアルトの気温は華氏 72 ドルで穏やかです。温度を摂氏に変換したい場合は、式 $c = \frac{5}{9}(f - 32)$ を評価し、$f$ を $72$ に設定します。この式では、値 $5$、$9$、および $32$ はスカラーです。変数 $c$ と $f$ は未知のスカラーを表します。

スカラーは通常の小文字 ($x$、$y$、$z$ など) で表し、すべての (連続した)*実数値*スカラーの空間は $\mathbb{R}$ で表します。便宜上、*スペース*の過去の厳密な定義は省略します。 $x \in \mathbb{R}$ という式は、$x$ が実数値のスカラーであることを正式に示す方法であることを覚えておいてください。記号 $\in$ (「イン」と発音) は、セット内のメンバーシップを示します。たとえば、$x, y \in {0, 1}$ は、$x$ と $y$ が値 $0$ または $1$ のみを取ることができる変数であることを示します。

 (**スカラーは 1 つの要素のみを含むテンソルとして実装されます。** ) 以下では、2 つのスカラーを割り当て、よく知られた加算、乗算、除算、およびべき乗演算を実行します。


In [2]:
x = torch.tensor(3.0)
y = torch.tensor(2.0)

x + y, x * y, x / y, x**y

(tensor(5.), tensor(6.), tensor(1.5000), tensor(9.))


## ベクトル

私たちの目的では、 [**ベクトルをスカラーの固定長配列と考えることができます。** ] 対応するコードと同様に、これらの値をベクトルの*要素*と呼びます (同義語には、*エントリ*と*コンポーネント*が含まれます)。ベクトルが現実世界のデータセットの例を表す場合、その値は現実世界の重要性を持ちます。たとえば、ローン不履行のリスクを予測するモデルをトレーニングする場合、各申請者を、収入、勤続年数、過去の債務不履行の回数などの量に対応する成分を持つベクトルに関連付けることができます。心臓発作のリスクを研究している場合、各ベクトルは患者を表し、その構成要素は患者の最新のバイタルサイン、コレステロール値、1 日あたりの運動時間などに対応する可能性があります。ベクトルは太字の小文字 (例: $\) で表します。 mathbf{x}$、$\mathbf{y}$、および $\mathbf{z}$)。

ベクトルは $1^{\mathrm{st}}$ 次のテンソルとして実装されます。一般に、このようなテンソルは、メモリの制限に従って任意の長さを持つことができます。注意: Python では、ほとんどのプログラミング言語と同様、ベクトル インデックスは $0$ (*ゼロベースのインデックス付け*とも呼ばれます) から始まりますが、線形代数の添字は $1$ (1 ベースのインデックス付け) から始まります。


In [3]:
x = torch.arange(3)
x

tensor([0, 1, 2])


添字を使用してベクトルの要素を参照できます。たとえば、$x_2$ は $\mathbf{x}$ の 2 番目の要素を示します。 $x_2$ はスカラーなので、太字にしません。デフォルトでは、要素を垂直に積み重ねることによってベクトルを視覚化します。

 $$\mathbf{x} =\begin{bmatrix}x_{1} \ \vdots \x_{n}\end{bmatrix},$$ :eqlabel: `eq_vec_def`

ここで、$x_1、\ldots、x_n$ はベクトルの要素です。後で、このような*列ベクトル*と要素が水平に積み上げられた*行ベクトル*を区別します。 [**インデックスを介してテンソルの要素にアクセスすることを思い出してください。** 】


In [4]:
x[2]

tensor(2)


ベクトルに $n$ 要素が含まれていることを示すには、 $\mathbf{x} \in \mathbb{R}^n$ と書きます。正式には $n$ をベクトルの*次元と*呼びます。 [**コードでは、これはテンソルの長さに対応します**]。Python の組み込み`len`関数を介してアクセスできます。


In [5]:
len(x)

3


`shape`属性を介して長さにアクセスすることもできます。この形状は、各軸に沿ったテンソルの長さを示すタプルです。 (**軸が 1 つだけのテンソルは、要素が 1 つだけの形状を持ちます。** )


In [6]:
x.shape

torch.Size([3])


多くの場合、「次元」という言葉は、軸の数と特定の軸に沿った長さの両方を意味するように多重定義されています。この混乱を避けるために、軸の数を参照する場合には*順序*を使用し、コンポーネントの数を参照する場合には*次元*数のみを使用します。

## 行列

スカラーが $0^{\mathrm{th}}$ 次のテンソルであり、ベクトルが $1^{\mathrm{st}}$ 次のテンソルであるのと同様に、行列は $2^{\mathrm{nd}}$ 次のテンソルです。行列は太字の大文字で表し ($\mathbf{X}$、$\mathbf{Y}$、$\mathbf{Z}$ など)、コードでは 2 つの軸を持つテンソルで表します。式 $\mathbf{A} \in \mathbb{R}^{m \times n}$ は、行列 $\mathbf{A}$ に $m \times n$ 個の実数値スカラーが $m として配置されていることを示します$ 行と $n$ 列。 $m = n$ の場合、行列は*正方行列*であると言います。視覚的には、任意の行列を表として示すことができます。個々の要素を参照するには、行インデックスと列インデックスの両方に添え字を付けます。たとえば、$a_{ij}$ は、$\mathbf{A}$ の $i^{\mathrm{th}}$ に属する値です。行と $j^{\mathrm{th}}$ 列:

 $$\mathbf{A}=\begin{bmatrix} a_{11} &amp; a_{12} &amp; \cdots &amp; a_{1n} \ a_{21} &amp; a_{22} &amp; \cdots &amp; a_{2n} \ \ vdots &amp; \vdots &amp; \ddots &amp; \vdots \ a_{m1} &amp; a_{m2} &amp; \cdots &amp; a_{mn} \ \end{bmatrix}.$$ :eqlabel: `eq_matrix_def`

コードでは、行列 $\mathbf{A} \in \mathbb{R}^{m \times n}$ を、形状 ($m$, $n$)。 [**適切なサイズの $m \times n$ テンソルを $m \times n$ 行列に変換できます**] 必要な形状を`reshape`に渡すことにより、次のようになります。


In [7]:
A = torch.arange(6).reshape(3, 2)
A

tensor([[0, 1],
        [2, 3],
        [4, 5]])


場合によっては、軸を反転したいことがあります。行列の行と列を交換すると、その結果は*転置*と呼ばれます。形式的には、行列 $\mathbf{A}$ の転置を $\mathbf{A}^\top$ で表し、 $\mathbf{B} = \mathbf{A}^\top$ の場合、 $b_{すべての $i$ と $j$ に対して ij} = a_{ji}$。したがって、 $m \times n$ 行列の転置は $n \times m$ 行列になります。

 $$ \mathbf{A}^\top = \begin{bmatrix} a_{11} &amp; a_{21} &amp; \dots &amp; a_{m1} \ a_{12} &amp; a_{22} &amp; \dots &amp; a_{m2 } \ \vdots &amp; \vdots &amp; \ddots &amp; \vdots \ a_{1n} &amp; a_{2n} &amp; \dots &amp; a_{mn} \end{bmatrix}。 $$

コードでは、次のように任意の (**行列の転置**) にアクセスできます。


In [8]:
A.T

tensor([[0, 2, 4],
        [1, 3, 5]])


[**対称行列は、独自の転置に等しい正方行列のサブセットです: $\mathbf{A} = \mathbf{A}^\top$。** ] 次の行列は対称です。


In [9]:
A = torch.tensor([[1, 2, 3], [2, 0, 4], [3, 4, 5]])
A == A.T

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])


行列はデータセットを表現するのに役立ちます。通常、行は個々のレコードに対応し、列は個別の属性に対応します。

## テンソル

スカラー、ベクトル、行列だけを使用して機械学習をさらに進めることができますが、最終的には高次の [**テンソル**] を使用する必要がある場合があります。テンソル ( **$n^{\mathrm{th}}$-order 配列の拡張を記述する一般的な方法を提供します。)***テンソル クラス*のソフトウェア オブジェクトを「テンソル」と呼ぶのは、これらも任意の数の軸を持つことができるためです。*テンソルという*言葉を数学的オブジェクトとコードでの実現の両方に使用するのは混乱を招くかもしれませんが、通常、その意味は文脈から明らかです。一般テンソルは、特殊なフォントフェイス ($\mathsf{X}$、$\mathsf{Y}$、$\mathsf{Z}$ など) とそのインデックス付けメカニズム ($x_{ など) を使用して大文字で表します。 ijk}$ と $[\mathsf{X}]_{1, 2i-1, 3}$) は行列の結果から自然に導き出されます。

画像を扱い始めると、テンソルがより重要になります。各画像は、高さ、幅、*チャネル*に対応する軸を持つ $3^{\mathrm{rd}}$ 次のテンソルとして到着します。各空間位置で、各色 (赤、緑、青) の強度がチャネルに沿って積み上げられます。さらに、画像のコレクションはコード内で $4^{\mathrm{th}}$-order tensor によって表され、個別の画像が最初の軸に沿ってインデックス付けされます。高次テンソルは、形状コンポーネントの数を増やすことによって、ベクトルや行列と同様に構築されます。


In [10]:
torch.arange(24).reshape(2, 3, 4)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])


## テンソル演算の基本特性

スカラー、ベクトル、行列、高次テンソルにはすべて、いくつかの便利な特性があります。たとえば、要素ごとの演算では、オペランドと同じ形状の出力が生成されます。


In [11]:
A = torch.arange(6, dtype=torch.float32).reshape(2, 3)
B = A.clone()  # Assign a copy of A to B by allocating new memory
A, A + B

(tensor([[0., 1., 2.],
         [3., 4., 5.]]),
 tensor([[ 0.,  2.,  4.],
         [ 6.,  8., 10.]]))


[**2 つの行列の要素ごとの積は、*アダマール積*** と呼ばれます] ($\odot$ で示されます)。以下に、2 つの行列 $\mathbf{A}, \mathbf{B} \in \mathbb{R}^{m \times n}$ のアダマール積のエントリを詳しく説明します。

 $$ \mathbf{A} \odot \mathbf{B} = \begin{bmatrix} a_{11} b_{11} &amp; a_{12} b_{12} &amp; \dots &amp; a_{1n} b_{1n} \ a_{21} b_{21} &amp; a_{22} b_{22} &amp; \dots &amp; a_{2n} b_{2n} \ \vdots &amp; \vdots &amp; \ddots &amp; \vdots \ a_{m1} b_{m1} &amp; a_{m2} b_{m2} &amp; \dots &amp; a_{mn} b_{mn} \end{bmatrix}。 $$


In [12]:
A * B

tensor([[ 0.,  1.,  4.],
        [ 9., 16., 25.]])


[**スカラーとテンソルを加算または乗算すると、**元のテンソルと同じ形状の結果が生成されます。ここで、テンソルの各要素はスカラーに加算 (または乗算) されます。


In [13]:
a = 2
X = torch.arange(24).reshape(2, 3, 4)
a + X, (a * X).shape

(tensor([[[ 2,  3,  4,  5],
          [ 6,  7,  8,  9],
          [10, 11, 12, 13]],
 
         [[14, 15, 16, 17],
          [18, 19, 20, 21],
          [22, 23, 24, 25]]]),
 torch.Size([2, 3, 4]))


## 割引

:label: `subsec_lin-alg-reduction`

多くの場合、**テンソルの要素の合計を計算したいことがあります。** ] 長さ $n$ のベクトル $\mathbf{x}$ の要素の合計を表すには、$\sum_{i=1}^n x_i$ と書きます。それには次のような簡単な関数があります。


In [14]:
x = torch.arange(3, dtype=torch.float32)
x, x.sum()

(tensor([0., 1., 2.]), tensor(3.))


[**任意の形状のテンソルの要素の合計**] を表現するには、その軸すべての合計を単純に計算します。たとえば、$m \times n$ 行列 $\mathbf{A}$ の要素の合計は $\sum_{i=1}^{m} \sum_{j=1}^{n} と書くことができます。 a_{ij}$。


In [15]:
A.shape, A.sum()

(torch.Size([2, 3]), tensor(15.))


デフォルトでは、関数 sum を呼び出すと、テンソルがすべての軸に沿って*縮小され*、最終的にスカラーが生成されます。私たちのライブラリでは、 [**テンソルを縮小する軸を指定することもできます。** ] 行 (軸 0) に沿ったすべての要素を合計するには、 `sum`に`axis=0`を指定します。入力行列は軸 0 に沿って縮小して出力ベクトルを生成するため、この軸は出力の形状から欠落しています。


In [16]:
A.shape, A.sum(axis=0).shape

(torch.Size([2, 3]), torch.Size([3]))


`axis=1`を指定すると、すべての列の要素が合計されるため、列の次元 (軸 1) が削減されます。


In [17]:
A.shape, A.sum(axis=1).shape

(torch.Size([2, 3]), torch.Size([2]))


合計によって行と列の両方に沿って行列を削減することは、行列のすべての要素を合計することと同じです。


In [18]:
A.sum(axis=[0, 1]) == A.sum() # Same as A.sum()

tensor(True)


[**関連する量は*平均*であり、*平均*とも呼ばれます。** ] 合計を要素の総数で割ることによって平均を計算します。平均値の計算は非常に一般的であるため、 `sum`と同様に機能する専用のライブラリ関数を取得します。


In [19]:
A.mean(), A.sum() / A.numel()

(tensor(2.5000), tensor(2.5000))


同様に、平均を計算する関数は、特定の軸に沿ってテンソルを削減することもできます。


In [20]:
A.mean(axis=0), A.sum(axis=0) / A.shape[0]

(tensor([1.5000, 2.5000, 3.5000]), tensor([1.5000, 2.5000, 3.5000]))


## 非減額合計

:label: `subsec_lin-alg-non-reduction`

合計または平均を計算する関数を呼び出すときに、[軸**の数を変更しない**] と便利な場合があります。これは、ブロードキャスト メカニズムを使用する場合に重要です。


In [21]:
sum_A = A.sum(axis=1, keepdims=True)
sum_A, sum_A.shape

(tensor([[ 3.],
         [12.]]),
 torch.Size([2, 1]))


たとえば、 `sum_A`各行を合計した後も 2 つの軸を保持するため、(**ブロードキャストを使用して`A`を`sum_A`で除算**)、各行の合計が $1$ になる行列を作成できます。


In [22]:
A / sum_A

tensor([[0.0000, 0.3333, 0.6667],
        [0.2500, 0.3333, 0.4167]])


[**ある軸に沿った`A`の要素の累積和**]、たとえば`axis=0` (行ごと) を計算したい場合は、 `cumsum`関数を呼び出すことができます。設計上、この関数はどの軸に沿っても入力テンソルを削減しません。


In [23]:
A.cumsum(axis=0)

tensor([[0., 1., 2.],
        [3., 5., 7.]])


## 内積

これまでは、要素ごとの演算、合計、平均のみを実行してきました。そして、これが私たちにできることのすべてであるならば、線形代数はそれ自身のセクションに値しないでしょう。幸いなことに、ここからがさらに興味深いことになります。最も基本的な演算の 1 つはドット積です。 2 つのベクトル $\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$ が与えられると、その*内積*$\mathbf{x}^\top \mathbf{y}$ (または $\langle \ mathbf{x}, \mathbf{y} \rangle$) は、同じ位置にある要素の積の合計です: $\mathbf{x}^\top \mathbf{y} = \sum_{i=1} ^{d} x_i y_i$。

 [~~2 つのベクトルの*内積は*、同じ位置にある要素の積の合計です~~]


In [24]:
y = torch.ones(3, dtype = torch.float32)
x, y, torch.dot(x, y)

(tensor([0., 1., 2.]), tensor([1., 1., 1.]), tensor(3.))


同様に、(**要素ごとの乗算とその後の合計を実行することで、2 つのベクトルの内積を計算できます。** )


In [25]:
torch.sum(x * y)

tensor(3.)


内積は幅広いコンテキストで役立ちます。たとえば、ベクトル $\mathbf{x} \in \mathbb{R}^n$ で示される値のセットと $\mathbf{w} \in \mathbb{R}^ で示される重みのセットが与えられたとします。 n$、重み $\mathbf{w}$ に従った $\mathbf{x}$ の値の加重和は、ドット積 $\mathbf{x}^\top \mathbf{w}$ として表すことができます。 。重みが負でなく、合計が 1 になる場合、つまり $\left(\sum_{i=1}^{n} {w_i} = 1\right)$ の場合、内積は*加重平均*を表します。 2 つのベクトルを単位長になるように正規化した後、内積はそれらの間の角度の余弦を表します。このセクションの後半で、この*長さ*の概念を正式に導入します。

## 行列ベクトル積

ドット積の計算方法がわかったので、$m \times n$ 行列 $\mathbf{A}$ と $n$ 次元ベクトル $\mathbf{x}$*の積を*理解し始めることができます。まず、行ベクトルの観点から行列を視覚化します。

 $$\mathbf{A}= \begin{bmatrix} \mathbf{a}^\top_{1} \ \mathbf{a}^\top_{2} \ \vdots \ \mathbf{a}^\top_m \ \終了{b行列}、$$

ここで、各 $\mathbf{a}^\top_{i} \in \mathbb{R}^n$ は、行列 $\mathbf{A}$ の $i^\mathrm{th}$ 行を表す行ベクトルです。 。

 **[ 行列とベクトルの積 $\mathbf{A}\mathbf{x}$ は単に長さ $m$ の列ベクトルであり、その $i^\mathrm{th}$ 要素はドット積 $\mathbf{a} です^\top_i \mathbf{x}$:** ]

 $$ \mathbf{A}\mathbf{x} = \begin{bmatrix} \mathbf{a}^\top_{1} \ \mathbf{a}^\top_{2} \ \vdots \ \mathbf{a} ^\top_m \ \end{bmatrix}\mathbf{x} = \begin{bmatrix} \mathbf{a}^\top_{1} \mathbf{x} \ \mathbf{a}^\top_{2} \mathbf {x} \ \vdots\ \mathbf{a}^\top_{m} \mathbf{x}\ \end{bmatrix}。 $$

行列 $\mathbf{A}\in \mathbb{R}^{m \times n}$ との乗算は、ベクトルを $\mathbb{R}^{n}$ から $\ に射影する変換として考えることができます。 mathbb{R}^{m}$。これらの変換は非常に便利です。たとえば、回転を特定の正方行列の乗算として表すことができます。行列ベクトル積は、前の層からの出力を考慮してニューラル ネットワークの各層の出力を計算する際に必要な主要な計算も記述します。



コードで行列とベクトルの積を表現するには、 `mv`関数を使用します。 `A`の列の次元 (軸 1 に沿った長さ) は、 `x`の次元 (その長さ) と同じである必要があることに注意してください。 PyTorch には、(引数に応じて) 行列-ベクトル積と行列-行列積の両方を実行できる便利な演算子`@`があります。したがって、 `A@x`と書くことができます。


In [26]:
A.shape, x.shape, torch.mv(A, x), A@x

(torch.Size([2, 3]), torch.Size([3]), tensor([ 5., 14.]), tensor([ 5., 14.]))


## 行列と行列の乗算

ドット積と行列ベクトル積のコツを理解していれば、*行列間の乗算は*簡単なはずです。

 2 つの行列 $\mathbf{A} \in \mathbb{R}^{n \times k}$ と $\mathbf{B} \in \mathbb{R}^{k \times m}$ があるとします。

 $$\mathbf{A}=\begin{bmatrix} a_{11} &amp; a_{12} &amp; \cdots &amp; a_{1k} \ a_{21} &amp; a_{22} &amp; \cdots &amp; a_{2k} \ \ vdots &amp; \vdots &amp; \ddots &amp; \vdots \ a_{n1} &amp; a_{n2} &amp; \cdots &amp; a_{nk} \ \end{bmatrix},\quad \mathbf{B}=\begin{bmatrix} b_{ 11} &amp; b_{12} &amp; \cdots &amp; b_{1m} \ b_{21} &amp; b_{22} &amp; \cdots &amp; b_{2m} \ \vdots &amp; \vdots &amp; \ddots &amp; \vdots \ b_{k1} &amp; b_{k2} &amp; \cdots &amp; b_{km} \ \end{bmatrix}.$$

 $\mathbf{a}^\top_{i} \in \mathbb{R}^k$ を行列 $\mathbf{A}$ の $i^\mathrm{th}$ 行を表す行ベクトルとします。 $\mathbf{b}_{j} \in \mathbb{R}^k$ が行列 $\mathbf{B}$ の $j^\mathrm{th}$ 列の列ベクトルを表すものとします。

 $$\mathbf{A}= \begin{bmatrix} \mathbf{a}^\top_{1} \ \mathbf{a}^\top_{2} \ \vdots \ \mathbf{a}^\top_n \ \ end{bmatrix}, \quad \mathbf{B}=\begin{bmatrix} \mathbf{b} *{1} &amp; \mathbf{b}* {2} &amp; \cdots &amp; \mathbf{b}_{m} \ \ {bmatrix} を終了します。 $$

行列積 $\mathbf{C} \in \mathbb{R}^{n \times m}$ を形成するには、単に各要素 $c_{ij}$ を $i^{\mathrm 間のドット積として計算します。 $\mathbf{A}$ の {th}}$ 行と $\mathbf{B}$ の $j^{\mathrm{th}}$ 列、つまり $\mathbf{a}^\top_i \mathbf {b}_j$:

 $$\mathbf{C} = \mathbf{AB} = \begin{bmatrix} \mathbf{a}^\top_{1} \ \mathbf{a}^\top_{2} \ \vdots \ \mathbf{a }^\top_n \ \end{bmatrix} \begin{bmatrix} \mathbf{b} *{1} &amp; \mathbf{b}* {2} &amp; \cdots &amp; \mathbf{b} *{m} \ \end{bmatrix} = \begin{bmatrix} \mathbf{a}^\top* {1} \mathbf{b} *1 &amp; \mathbf{a}^\top* {1}\mathbf{b} *2&amp; \cdots &amp; \mathbf{a}^ \top* {1} \mathbf{b} *m \ \mathbf{a}^\top* {2}\mathbf{b} *1 &amp; \mathbf{a}^\top* {2} \mathbf{b} *2 &amp; \cdots &amp; \mathbf{a}^\top* {2} \mathbf{b} *m \ \vdots &amp; \vdots &amp; \ddots &amp;\vdots\ \mathbf{a}^\top* {n} \mathbf{b} *1 &amp; \ mathbf{a}^\top* {n}\mathbf{b} *2&amp; \cdots&amp; \mathbf{a}^\top* {n} \mathbf{b}_m \end{bmatrix}。 $$

 **[ 行列と行列の乗算 $\mathbf{AB}$ は、 $m$ 行列ベクトル積または $m \times n$ ドット積を実行し、結果をつなぎ合わせて $n \times m$ 行列を形成すると考えることができます。** ] 次のスニペットでは、 `A`と`B`に対して行列の乗算を実行します。ここで、 `A`は2行3列の行列、 `B`は3行4列の行列です。乗算後、2 行 4 列の行列が得られます。


In [27]:
B = torch.ones(3, 4)
torch.mm(A, B), A@B

(tensor([[ 3.,  3.,  3.,  3.],
         [12., 12., 12., 12.]]),
 tensor([[ 3.,  3.,  3.,  3.],
         [12., 12., 12., 12.]]))


*行列間の乗算という*用語は、多くの場合、*行列乗算*に簡略化されるため、アダマール積と混同しないでください。

## 規範

:label: `subsec_lin-algebra-norms`

線形代数で最も役立つ演算子の一部は、*ノルム*です。非公式には、ベクトルのノルムによって、そのベクトルの*大きさ*がわかります。たとえば、$\ell_2$ ノルムはベクトルの (ユークリッド) 長さを測定します。ここでは、ベクトルのコンポーネント (次元ではなく) の大きさに関係する*サイズ*の概念を採用しています。

ノルムは関数です $| \cdot |$ は、ベクトルをスカラーにマップし、次の 3 つのプロパティを満たします。
1. 任意のベクトル $\mathbf{x}$ が与えられた場合、そのベクトル (のすべての要素) をスカラー $\alpha \in \mathbb{R}$ でスケーリングすると、そのノルムはそれに応じてスケーリングされます: $$|\alpha \mathbf{x }| = |\アルファ| |\mathbf{x}|.$$
1. 任意のベクトル $\mathbf{x}$ および $\mathbf{y}$ について: ノルムは三角不等式を満たす: $$|\mathbf{x} + \mathbf{y}| \leq |\mathbf{x}| + |\mathbf{y}|.$$
1. ベクトルのノルムは非負であり、ベクトルが 0 の場合にのみ消滅します。 $$|\mathbf{x}| &gt; 0 \text{ すべての } \mathbf{x} \neq 0.$$

多くの関数は有効な標準であり、異なる標準は異なるサイズの概念をエンコードします。小学校の幾何学で直角三角形の斜辺を計算するときに誰もが習ったユークリッドノルムは、ベクトルの要素の二乗和の平方根です。正式には、これは [**the $\ell_2$ *norm***] と呼ばれ、次のように表現されます。

 ( **$$|\mathbf{x}| *2 = \sqrt{\sum* {i=1}^n x_i^2}.$$** )

メソッド`norm` $\ell_2$ ノルムを計算します。


In [28]:
u = torch.tensor([3.0, -4.0])
torch.norm(u)

tensor(5.)


[ **$\ell_1$ ノルム**] も人気があり、関連する指標はマンハッタン距離と呼ばれます。定義により、$\ell_1$ ノルムはベクトルの要素の絶対値を合計します。

 ( **$$|\mathbf{x}| *1 = \sum* {i=1}^n \left|x_i \right|.$$** )

 $\ell_2$ ノルムと比較して、外れ値に対する感度が低くなります。 $\ell_1$ ノルムを計算するには、合計演算で絶対値を合成します。


In [29]:
torch.abs(u).sum()

tensor(7.)


$\ell_2$ ノルムと $\ell_1$ ノルムはどちらも、より一般的な $\ell_p$*ノルム*の特殊なケースです。

 $$|\mathbf{x}| *p = \left(\sum* {i=1}^n \left|x_i \right|^p \right)^{1/p}.$$

行列の場合、問題はさらに複雑になります。結局のところ、行列は個々のエントリのコレクションとして*、また*ベクトルを操作して他のベクトルに変換するオブジェクトとして見ることができます。たとえば、行列とベクトルの積 $\mathbf{X} \mathbf{v}$ が $\mathbf{v}$ に対してどのくらい長くなり得るかを尋ねることができます。この考え方は、*スペクトル*規範と呼ばれる規範につながります。ここでは、[**計算がはるかに簡単な*フロベニウス ノルム***] を導入し、行列の要素の二乗和の平方根として定義します。

 [ **$$|\mathbf{X}| *F = \sqrt{\sum* {i=1}^m \sum_{j=1}^n x_{ij}^2}.$$ ]**

フロベニウス ノルムは、行列状ベクトルの $\ell_2$ ノルムであるかのように動作します。次の関数を呼び出すと、行列のフロベニウス ノルムが計算されます。


In [30]:
torch.norm(torch.ones((4, 9)))

tensor(6.)


あまり先を行きたくありませんが、これらの概念がなぜ役立つのかについて、すでに直感を植え付けることはできます。深層学習では、多くの場合、観測データに割り当てられる確率*を最大化するという*最適化問題を解決しようとします。レコメンダーモデルに関連する収益*を最大化します*。予測と真実の観察との間の距離*を最小限に抑える*。同じ人物の写真の表現間の距離*を最小限に抑え*ながら、異なる人物の写真の表現間の距離*を最大化します*。深層学習アルゴリズムの目的を構成するこれらの距離は、多くの場合、規範として表現されます。

## 議論

このセクションでは、現代の深層学習の注目すべき部分を理解するために必要なすべての線形代数を確認しました。線形代数にはさらに多くの機能があり、その多くは機械学習に役立ちます。たとえば、行列は因子に分解でき、これらの分解によって現実世界のデータセットの低次元構造が明らかになります。機械学習には、データセット内の構造を発見し、予測問題を解決するために、行列分解とその高次テンソルへの一般化を使用することに焦点を当てたサブフィールド全体があります。しかし、この本はディープラーニングに焦点を当てています。そして、実際のデータセットに機械学習を適用してみると、さらに数学を学ぶ傾向が高まると私たちは信じています。したがって、後でさらに数学を紹介する権利を留保しますが、このセクションはここで終了します。

線形代数をさらに学びたい場合は、優れた書籍やオンライン リソースがたくさんあります。より高度な集中コースについては、 :citet: `Strang.1993,Kolter.2008,Petersen.Pedersen.ea.2008`をチェックすることを検討してください。

要点をまとめると：
- スカラー、ベクトル、行列、およびテンソルは、線形代数で使用される基本的な数学オブジェクトであり、それぞれ 0、1、2、および任意の数の軸を持ちます。
- テンソルは、インデックス付け、または`sum`や`mean`などの演算によって、それぞれ指定された軸に沿ってスライスまたは削減できます。
-  Elementwise積はアダマール積と呼ばれます。対照的に、ドット積、行列とベクトルの積、および行列と行列の積は要素ごとの演算ではなく、一般にオペランドとは異なる形状を持つオブジェクトを返します。
- アダマール積と比較すると、行列間積の計算にはかなり長い時間がかかります (2 次時間ではなく 3 次時間)。
- ノルムはベクトルの大きさに関するさまざまな概念を捉えており、通常は 2 つのベクトルの差に適用されて、それらの距離を測定します。
- 一般的なベクトル ノルムには $\ell_1$ ノルムと $\ell_2$ ノルムが含まれ、一般的な行列ノルムには*スペクトル ノルム*と*フロベニウス*ノルムが含まれます。

## 演習
1. 行列の転置の転置が行列そのものであることを証明します: $(\mathbf{A}^\top)^\top = \mathbf{A}$。
1.  2 つの行列 $\mathbf{A}$ と $\mathbf{B}$ が与えられた場合、和と転置が交換できることを示します。 $\mathbf{A}^\top + \mathbf{B}^\top = (\mathbf{A } + \mathbf{B})^\top$。
1. 正方行列 $\mathbf{A}$ が与えられた場合、$\mathbf{A} + \mathbf{A}^\top$ は常に対称ですか?前の 2 つの演習の結果のみを使用して結果を証明できますか?
1. このセクションでは、形状 (2, 3, 4) のテンソル`X`を定義しました。 `len(X)`の出力は何ですか?コードを実装せずに答えを書き、コードを使用して答えを確認します。
1. 任意の形状のテンソル`X`の場合、 `len(X)`常に`X`の特定の軸の長さに対応しますか?その軸とは何でしょうか？
1.  `A / A.sum(axis=1)`を実行して、何が起こるかを確認します。理由を分析できますか？
1. マンハッタンのダウンタウンの 2 点間を移動する場合、座標、つまり大通りや道路の観点からカバーする必要がある距離はどれくらいですか?斜めに移動できますか？
1. 形状 (2, 3, 4) のテンソルを考えてみましょう。軸 0、1、2 に沿った合計出力の形状は何ですか?
1.  3 軸以上のテンソルを関数`linalg.norm`に供給し、その出力を観察します。この関数は、任意の形状のテンソルに対して何を計算しますか?
1.  3 つの大きな行列を定義します。$\mathbf{A} \in \mathbb{R}^{2^{10} \times 2^{16}}$、$\mathbf{B} \in \mathbb{R}^ とします。 {2^{16} \times 2^{5}}$ と $\mathbf{C} \in \mathbb{R}^{2^{5} \times 2^{14}}$ は、たとえば次のように初期化されます。ガウス確率変数。積 $\mathbf{A} \mathbf{B} \mathbf{C}$ を計算したいとします。 $(\mathbf{A} \mathbf{B}) \mathbf{C}$ を計算するか $\mathbf{A} (\mathbf{B} \mathbf{ C})$。なぜ？
1.  3 つの大きな行列を定義します。$\mathbf{A} \in \mathbb{R}^{2^{10} \times 2^{16}}$、$\mathbf{B} \in \mathbb{R}^ とします。 {2^{16} \times 2^{5}}$ と $\mathbf{C} \in \mathbb{R}^{2^{5} \times 2^{16}}$。 $\mathbf{A} \mathbf{B}$ を計算するか $\mathbf{A} \mathbf{C}^\top$ を計算するかによって速度に違いはありますか?なぜ？メモリのクローンを作成せずに $\mathbf{C} = \mathbf{B}^\top$ を初期化すると何が変わるでしょうか?なぜ？
1.  3 つの行列、たとえば $\mathbf{A}、\mathbf{B}、\mathbf{C} \in \mathbb{R}^{100 \times 200}$ を定義します。 $[\mathbf{A}, \mathbf{B}, \mathbf{C}]$ を積み上げて 3 軸のテンソルを構成します。次元とは何ですか？ 3 番目の軸の 2 番目の座標をスライスして $\mathbf{B}$ を復元します。答えが正しいかどうかを確認してください。



[ディスカッション](https://discuss.d2l.ai/t/31)
